In [ ]:
from torch.utils import data

In [ ]:
# torch.utils.data.Dataset
# 可自定义数据集
# custom_dataset = CustomDataset() 
# CustomDataset(torch.utils.data.Dataset), 需定义好__init__(self), __getitem__(self, index)和__len__(self)

# torch.utils.data.TensorDataset(data_tensor, target_tensor) # 包装数据和目标张量的数据集

In [ ]:
# 计算数据集的均值和标准差
# 输入PyTorch的dataset, 输出均值和标准差
import numpy as np
def compute_mean_and_std(dataset):
    mean_r = 0
    mean_g = 0
    mean_b = 0
    for img, _ in dataset:
        img = np.asarray(img) # change PIL Image to numpy array
        mean_b += np.mean(img[:, :, 0])
        mean_g += np.mean(img[:, :, 1])
        mean_r += np.mean(img[:, :, 2])

    mean_b /= len(dataset)
    mean_g /= len(dataset)
    mean_r /= len(dataset)

    diff_r = 0
    diff_g = 0
    diff_b = 0

    N = 0

    for img, _ in dataset:
        img = np.asarray(img)

        diff_b += np.sum(np.power(img[:, :, 0] - mean_b, 2))
        diff_g += np.sum(np.power(img[:, :, 1] - mean_g, 2))
        diff_r += np.sum(np.power(img[:, :, 2] - mean_r, 2))

        N += np.prod(img[:, :, 0].shape)

    std_b = np.sqrt(diff_b / N)
    std_g = np.sqrt(diff_g / N)
    std_r = np.sqrt(diff_r / N)

    mean = (mean_b.item() / 255.0, mean_g.item() / 255.0, mean_r.item() / 255.0)
    std = (std_b.item() / 255.0, std_g.item() / 255.0, std_r.item() / 255.0)
    return mean, std

In [ ]:
train_set = None
train_set = data.Subset(train_set, train_idx_normal=[])
# 函数参数(dataset, indices): 获取指定一个索引序列对应的子数据集   
# 返回一个'Subset'对象

In [ ]:
dataset = None
train_loader = data.DataLoader(dataset, 4) # 以简单的方式提供队列和线程
# 第一个参数类型为torch.utils.data.Dataset, 第二个参数为batch_size
# 还有参数num_workers, shuffle(训练集置为True), pin_memory, drop_last
	# 以及batch_sampler, collate_fn, sampler(当该参数为空时, 一般pin_memory=False)
# pin_memory 
	# torch.utils.data.DataLoader中尽量设置pin_memory=True
	# 对特别小的数据集如MNIST设置pin_memory=False反而更快一些
# num_workers
	# 设置需要在实验中找到最快的取值
	# num_workers的经验设置值是自己电脑/服务器的CPU核心数, 如果CPU很强, RAM也很充足, 就可以设置得更大些
# drop_last
	# 如果数据集大小不能被batch size整除, 则设置为True后可删除最后一个不完整的batch
	# 如果设为False并且数据集的大小不能被batch size整除, 则最后一个batch将更小
	# 训练使用的数据集, 开启drop_last, 这样如果最后一个batch小于你的batch_size, 会扔掉, 训练会更稳定
# batch_sampler
	# 可来自BucketBatchSampler
len(train_loader.dataset)  # 数据点的个数
data_iter = iter(train_loader) # 当迭代开始时, 队列和线程开始从文件下载数据
images, labels = data_iter.next() # Mini-batch

In [ ]:
# torch.utils.data.sampler.Sampler
# 一个迭代器基类
	# def __init__(self, data_source):
    #     pass

    # def __iter__(self):
    #     raise NotImplementedError

    # def __len__(self):
    #     raise NotImplementedError
# SequentialSampler
	# 按顺序对数据集采样
	# 在__iter__方法中首先得到一个和data_source一样长度的range可迭代器; 每次只会返回一个索引值
	# 即iter(range(len(self.data_source)))
# RandomSampler: 
	# num_samples: 指定采样的数量, 默认是所有
	# replacement: 若为True, 则表示可以重复采样, 即同一个样本可以重复采样, 这样可能导致有的样本采样不到; 所以此时我们可以设置num_samples来增加采样数量使得每个样本都可能被采样到
	# __iter__方法
		# n = len(self.data_source)
		# if self.replacement:
		# 	return iter(torch.randint(high=n, size=(self.num_samples,), dtype=torch.int64).tolist())
		# return iter(torch.randperm(n).tolist())
# SubsetRandomSampler
	# 这个采样器常见的使用场景是将训练集划分成训练集和验证集 
	# init函数中给定indices
	# iter函数返回(self.indices[i] for i in torch.randperm(len(self.indices)))
# WeightedRandomSampler
	# init函数中多了参数weights
	# self.weights = torch.tensor(weights, dtype=torch.double) 或 self.weights = torch.DoubleTensor(weights)
	# iter函数返回iter(torch.multinomial(self.weights, self.num_samples, self.replacement=True).tolist())